In [ ]:
import geopandas as gpd
import numpy as np
from osmnx import geocode_to_gdf
import plotly.express as px
from shapely.geometry import MultiPolygon, Point, Polygon

from srai.regionizers import VoronoiRegionizer

## Regionize whole Earth

Basic usage of `VoronoiRegionizer` to cover whole Earth using 6 poles.

In [ ]:
# 6 poles of the Earth
seeds_gdf = gpd.GeoDataFrame(
    {
        "geometry": [
            Point(0, 0),
            Point(90, 0),
            Point(180, 0),
            Point(-90, 0),
            Point(0, 90),
            Point(0, -90),
        ]
    },
    index=[1, 2, 3, 4, 5, 6],
    crs="EPSG:4326",
)

In [ ]:
seeds_gdf.plot()

In [ ]:
vr = VoronoiRegionizer(seeds=seeds_gdf)

In [ ]:
result_gdf = vr.transform()

In [ ]:
result_gdf

In [ ]:
fig = px.choropleth(
    result_gdf,
    geojson=result_gdf.geometry,
    locations=result_gdf.index,
    color=result_gdf.index,
    color_continuous_scale=px.colors.sequential.Viridis,
)
fig2 = px.scatter_geo(seeds_gdf, lat=seeds_gdf.geometry.y, lon=seeds_gdf.geometry.x)
fig.update_traces(marker={"opacity": 0.6}, selector=dict(type="choropleth"))
fig.add_trace(fig2.data[0])
fig.update_traces(marker_color="white", marker_size=10, selector=dict(type="scattergeo"))
fig.update_layout(coloraxis_showscale=False)
fig.update_geos(
    projection_type="orthographic", projection_rotation_lon=20, projection_rotation_lat=30
)
fig.update_layout(height=800, width=800, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show(renderer="png")  # replace with fig.show() to allow interactivity

## Regionize single country

Drawing a list of arbitrary points inside of the country boundary and using them for regionization of the same geometry.

In [ ]:
uk_gdf = geocode_to_gdf(query=["R62149"], by_osmid=True)

uk_gdf = uk_gdf.to_crs(epsg=4326)  # convert to wgs84
uk_gdf_shape = uk_gdf.iloc[0].geometry  # get the Polygon

In [ ]:
uk_gdf

In [ ]:
N_POINTS = 100
# generate some random points within the bounds
minx, miny, maxx, maxy = uk_gdf_shape.bounds

randx = np.random.uniform(minx, maxx, N_POINTS)
randy = np.random.uniform(miny, maxy, N_POINTS)
coords = np.vstack((randx, randy)).T

# use only the points inside the geographic area

pts = [p for p in list(map(Point, coords)) if p.within(uk_gdf_shape)]

uk_seeds_gdf = gpd.GeoDataFrame(
    {"geometry": pts},
    index=list(range(len(pts))),
    crs="EPSG:4326",
)

del coords  # not used any more

In [ ]:
uk_seeds_gdf.plot()

In [ ]:
vr_uk = VoronoiRegionizer(seeds=uk_seeds_gdf)

In [ ]:
uk_result_gdf = vr_uk.transform(gdf=uk_gdf)

In [ ]:
uk_result_gdf.head()

In [ ]:
fig = px.choropleth(
    uk_result_gdf,
    geojson=uk_result_gdf.geometry,
    locations=uk_result_gdf.index,
    color=uk_result_gdf.index,
    color_continuous_scale=px.colors.qualitative.Plotly,
)
fig2 = px.scatter_geo(uk_seeds_gdf, lat=uk_seeds_gdf.geometry.y, lon=uk_seeds_gdf.geometry.x)
fig.update_traces(marker={"opacity": 0.6}, selector=dict(type="choropleth"))
fig.add_trace(fig2.data[0])
fig.update_traces(marker_color="black", marker_size=6, selector=dict(type="scattergeo"))
fig.update_layout(coloraxis_showscale=False)
fig.update_geos(
    projection_type="natural earth",
    lataxis_range=[miny - 1, maxy + 1],
    lonaxis_range=[minx - 1, maxx + 1],
    resolution=50,
    showframe=False,
    showlakes=False,
)
fig.update_layout(height=800, width=675, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show(renderer="png")  # replace with fig.show() to allow interactivity

## Higher amount of points

Example of railway stations in Germany (5000+ seeds) with multiprocessing.

In [ ]:
stations_csv = gpd.pd.read_csv('https://raw.githubusercontent.com/trainline-eu/stations/master/stations.csv', sep=';', index_col='id', usecols=['id', 'latitude', 'longitude', 'country'])
stations_csv

In [ ]:
stations = []
positions = set()
for idx, r in stations_csv.iterrows():
    if r.country != 'DE' or gpd.pd.isna(r.latitude) or gpd.pd.isna(r.longitude):
        continue
    pos = round(r.longitude,5), round(r.latitude,5)
    if not pos in positions:
        stations.append({"id": idx, "geometry": Point(*pos)})
        positions.add(pos)

stations_gdf = gpd.GeoDataFrame(data=stations, crs="EPSG:4326").set_index('id')

del stations_csv
del stations
del positions

stations_gdf.head()

In [ ]:
vr_rail = VoronoiRegionizer(seeds=stations_gdf, allow_multiprocessing=True)

In [ ]:
rail_result_gdf = vr_rail.transform()

In [ ]:
# Germany
fig = px.choropleth(
    rail_result_gdf,
    geojson=rail_result_gdf.geometry,
    locations=rail_result_gdf.index,
    color=rail_result_gdf.index,
    color_continuous_scale=px.colors.sequential.Viridis,
)
fig2 = px.scatter_geo(
    stations_gdf, lat=stations_gdf.geometry.y, lon=stations_gdf.geometry.x
)
fig.update_traces(marker={"opacity": 0.6}, selector=dict(type="choropleth"))
fig.add_trace(fig2.data[0])
fig.update_traces(marker_color="white", marker_size=2, selector=dict(type="scattergeo"))
fig.update_layout(coloraxis_showscale=False)
fig.update_geos(
    projection_type="orthographic",
    projection_rotation_lon=10,
    projection_rotation_lat=51,
    projection_scale=11,
    resolution=50,
    showlakes=False,
    showcountries=True,
    showframe=False,
)
fig.update_layout(height=800, width=800, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show(renderer="png")  # replace with fig.show() to allow interactivity

In [ ]:
# Berlin
fig = px.choropleth_mapbox(
    rail_result_gdf,
    geojson=rail_result_gdf,
    color=rail_result_gdf.index,
    locations=rail_result_gdf.index,
    center={"lat": 52.51637, "lon": 13.40665},
    mapbox_style="open-street-map",
    zoom=11,
)
fig2 = px.scatter_mapbox(
    stations_gdf, lat=stations_gdf.geometry.y, lon=stations_gdf.geometry.x
)
fig.add_trace(fig2.data[0])
fig.update_layout(margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.update_layout(coloraxis_showscale=False)
fig.update_traces(marker={"opacity": 0.6}, selector=dict(type="choroplethmapbox"))
fig.update_traces(marker_color="white", marker_size=5, selector=dict(type="scattermapbox"))
fig.update_layout(height=800, width=800, margin={"r": 0, "t": 0, "l": 0, "b": 0})
fig.show(renderer="png")  # replace with fig.show() to allow interactivity

## Difference between spherical voronoi and 2d voronoi

Showing the difference between working on the sphere and projected 2D plane.

Uses `geovoronoi` package as an example.

In [ ]:
"""
Geovoronoi package allows for a quick division of the Earth using list of seeds on a projected 2d plane.
This results in straight lines with angles distorted and polygons differences
might be substantial during comparisons or any area calculations.
"""
# geovoronoi package isn't used in this library, but is optional and can be installed using
# pip install geovoronoi
from geovoronoi import voronoi_regions_from_coords

from shapely.geometry.polygon import orient
from plotly.subplots import make_subplots

In [ ]:
pl_gdf = geocode_to_gdf(query=["R49715"], by_osmid=True)

pl_gdf = pl_gdf.to_crs(epsg=4326)  # convert to wgs84
pl_gdf_shape = pl_gdf.iloc[0].geometry  # get the Polygon

In [ ]:
N_POINTS = 100
# generate some random points within the bounds
minx, miny, maxx, maxy = pl_gdf_shape.bounds

randx = np.random.uniform(minx, maxx, N_POINTS)
randy = np.random.uniform(miny, maxy, N_POINTS)
coords = np.vstack((randx, randy)).T

# use only the points inside the geographic area

pts = [p for p in list(map(Point, coords)) if p.within(pl_gdf_shape)]

pl_seeds_gdf = gpd.GeoDataFrame(
    {"geometry": pts},
    index=list(range(len(pts))),
    crs="EPSG:4326",
)

del coords

In [ ]:
region_polys, region_pts, unassigned_pts = voronoi_regions_from_coords(
    pts, pl_gdf_shape, return_unassigned_points=True, per_geom=False
)

In [ ]:
def orient_geom(geom):
    if type(geom) == Polygon:
        return orient(geom, sign=-1)
    elif type(geom) == MultiPolygon:
        return MultiPolygon([orient(g, sign=-1) for g in geom.geoms])


pl_regions_2d_gdf = gpd.GeoDataFrame(
    {"geometry": [orient_geom(geom) for geom in region_polys.values()]},
    index=list(range(len(region_polys))),
    crs="EPSG:4326",
)

In [ ]:
pl_regions_2d_gdf

In [ ]:
vr_pl = VoronoiRegionizer(seeds=pl_seeds_gdf)

In [ ]:
pl_result_gdf = vr_pl.transform(gdf=pl_gdf)

In [ ]:
pl_result_gdf

In [ ]:
choropleth_1 = px.choropleth(
    pl_result_gdf,
    geojson=pl_result_gdf.geometry,
    locations=pl_result_gdf.index,
    color=pl_result_gdf.index,
    color_continuous_scale=px.colors.qualitative.Plotly,
)

choropleth_2 = px.choropleth(
    pl_regions_2d_gdf,
    geojson=pl_regions_2d_gdf.geometry,
    locations=pl_regions_2d_gdf.index,
    color=pl_regions_2d_gdf.index,
    color_continuous_scale=px.colors.qualitative.Plotly,
)

points_plot = px.scatter_geo(pl_seeds_gdf, lat=pl_seeds_gdf.geometry.y, lon=pl_seeds_gdf.geometry.x)

fig = make_subplots(
    rows=2,
    cols=2,
    specs=[
        [{"type": "scattergeo"}, {"type": "scattergeo"}],
        [{"type": "scattergeo"}, {"type": "scattergeo"}],
    ],
    horizontal_spacing=0.01,
    vertical_spacing=0.0,
)

fig.add_trace(choropleth_1["data"][0], row=1, col=1)
fig.add_trace(choropleth_1["data"][0], row=2, col=1)
fig.add_trace(choropleth_2["data"][0], row=1, col=2)
fig.add_trace(choropleth_2["data"][0], row=2, col=2)
for r in [1, 2]:
    for c in [1, 2]:
        fig.add_trace(points_plot.data[0], row=r, col=c)

fig.update_traces(marker_color="black", marker_size=6, selector=dict(type="scattergeo"))
fig.update_layout(coloraxis_showscale=False)
fig.update_geos(
    projection_type="natural earth",
    lataxis_range=[miny - 1, maxy + 1],
    lonaxis_range=[minx - 1, maxx + 1],
    resolution=50,
    row=1,
)

fig.update_geos(
    projection_type="natural earth",
    lataxis_range=[miny + 1, maxy - 1],
    lonaxis_range=[minx + 2, maxx - 2],
    resolution=50,
    row=2,
)

fig.update_traces(marker={"opacity": 0.6}, selector=dict(type="choropleth"), row=1)
fig.update_traces(marker={"opacity": 0.25}, selector=dict(type="choropleth"), row=2)

fig.update_layout(
    height=800,
    width=800,
    title_text="Side By Side Subplots (Left: Spherical voronoi, Right: 2D voronoi)",
    margin={"r": 5, "t": 50, "l": 5, "b": 0},
)
fig.show(renderer="png")  # replace with fig.show() to allow interactivity